<div style="display: flex; background-color: RGB(255, 255, 240);" >
<h1 style="margin: auto; padding: 30px; ">SCRIPT DE DETECTION DE FAUX BILLETS </h1>
</div>

In [2]:
#Import des librairies
import joblib
import pandas as pd
import numpy as np

<div style="border: 1px solid RGB(51,165,182);" >
<h3 style="margin: auto; padding: 20px; color: RGB(51,165,182); ">1. CHARGEMENT DES OUTILS ET MODÈLES</h3>
</div>

In [5]:
# Charger le modèle et le scaler sauvegardés
modele_rf_charge = joblib.load('modele_random_forest_final.joblib')
scaler_charge = joblib.load('scaler_standard.joblib')

In [6]:
# Définir l'ordre des caractéristiques pour le modèle
FEATURES = ['diagonal', 'height_left', 'height_right', 'margin_low', 'margin_up', 'length']

<div style="border: 1px solid RGB(51,165,182);" >
<h3 style="margin: auto; padding: 20px; color: RGB(51,165,182); ">2. FONCTION DE PRÉDICTION</h3>
</div>

In [10]:
def tester_avec_csv(chemin_fichier_test):
    """
    Charge un fichier CSV de données brutes, applique le même nettoyage (suppression des NaN) 
    que pour l'entraînement, et fait la prédiction.
    """
    print(f"Chargement et nettoyage des données depuis : {chemin_fichier_test}")
    
    # 1. Charger le CSV (df_raw est le dataframe brut, y compris les NaN si présents)
    try:
        df_raw = pd.read_csv(chemin_fichier_test)
    except FileNotFoundError:
        print(f"Erreur : Le fichier {chemin_fichier_test} n'a pas été trouvé. Veuillez vérifier le chemin.")
        return None

    # S'assurer que le DataFrame contient bien les 6 colonnes
    if not all(feature in df_raw.columns for feature in FEATURES):
        print("Erreur : Le fichier CSV ne contient pas toutes les colonnes requises.")
        return None
    
    # ÉTAPE CRUCIALE : REPRODUCTION DU NETTOYAGE
    lignes_avant = len(df_raw)
    
    # df_clean_pour_pred est le DataFrame PROPRE utilisé pour toutes les étapes suivantes
    df_clean_pour_pred = df_raw.dropna(subset=FEATURES).copy()
    lignes_apres = len(df_clean_pour_pred)
    
    if lignes_avant != lignes_apres:
        print(f"INFO: {lignes_avant - lignes_apres} lignes avec des valeurs manquantes ont été supprimées.")
    
    if df_clean_pour_pred.empty:
        print("Erreur : Après nettoyage, le fichier ne contient plus de lignes valides pour la prédiction.")
        return None
    
    # Isoler les données à prédire, en garantissant l'ordre
    X_pred = df_clean_pour_pred[FEATURES]
    
    # 2. Mise à l'échelle des données de test
    X_pred_array = scaler_charge.transform(X_pred)
    
    # Reconvertir en DataFrame AVEC les noms de colonnes (pour éviter les UserWarnings)
    X_pred_scaled_df = pd.DataFrame(X_pred_array, columns=FEATURES)

    # 3. Prédiction
    predictions = modele_rf_charge.predict(X_pred_scaled_df) 
    probabilities = modele_rf_charge.predict_proba(X_pred_scaled_df)
    
    # 4. Intégrer les résultats au DataFrame nettoyé
    df_clean_pour_pred['Prediction'] = predictions
    df_clean_pour_pred['Probabilite_Faux (%)'] = probabilities[:, 1] * 100
    df_clean_pour_pred['Resultat'] = np.where(predictions == 1, 'FAUX', 'VRAI') 

    return df_clean_pour_pred


<div style="border: 1px solid RGB(51,165,182);" >
<h3 style="margin: auto; padding: 20px; color: RGB(51,165,182); ">3. EXECUTION DE LA PREDICTION</h3>
</div>

In [38]:
chemin_du_csv_a_tester = 'df_clean.csv' 

# L'exécution de la fonction crée le DataFrame des résultats
resultats_df = tester_avec_csv(chemin_du_csv_a_tester)

if resultats_df is not None:
    print("\n--- RÉSULTATS DE LA CLASSIFICATION ---")
    # Afficher les 5 premières lignes du tableau complet
    print(resultats_df.head().to_markdown(index=False)) 
    
    print("\n--- SYNTHÈSE DES PRÉDICTIONS ---")
    # Afficher le nombre total de 'VRAI' et de 'FAUX' prédits
    prediction_counts = resultats_df['Resultat'].value_counts()
    print(prediction_counts.to_markdown())

Chargement et nettoyage des données depuis : df_clean.csv

--- RÉSULTATS DE LA CLASSIFICATION ---
| is_genuine   |   diagonal |   height_left |   height_right |   margin_low |   margin_up |   length | is_genuine_str   |   Prediction |   Probabilite_Faux (%) | Resultat   |
|:-------------|-----------:|--------------:|---------------:|-------------:|------------:|---------:|:-----------------|-------------:|-----------------------:|:-----------|
| True         |     171.81 |        104.86 |         104.95 |         4.52 |        2.89 |   112.83 | Vrai             |            0 |                      9 | VRAI       |
| True         |     171.46 |        103.36 |         103.66 |         3.77 |        2.99 |   113.09 | Vrai             |            0 |                      1 | VRAI       |
| True         |     172.69 |        104.48 |         103.5  |         4.4  |        2.94 |   113.16 | Vrai             |            0 |                      0 | VRAI       |
| True         |     171.36

C:\Users\elodi\anaconda3\Lib\site-packages\sklearn\base.py:486: UserWarning: X has feature names, but RandomForestClassifier was fitted without feature names
  warnings.warn(
C:\Users\elodi\anaconda3\Lib\site-packages\sklearn\base.py:486: UserWarning: X has feature names, but RandomForestClassifier was fitted without feature names
  warnings.warn(


In [40]:
if resultats_df is not None:
    # ----------------------------------------------------
    # NOUVEAU : Export du DataFrame complet dans un fichier Excel
    # ----------------------------------------------------
    nom_fichier_export_excel = 'resultats_classification_complets.xlsx'
    
    # Exporte le DataFrame dans un fichier Excel
    resultats_df.to_excel(nom_fichier_export_excel, index=False)
    
    print(f"\n✅ EXPORT RÉUSSI : Les résultats complets ont été sauvegardés dans le fichier : {nom_fichier_export_excel}")


✅ EXPORT RÉUSSI : Les résultats complets ont été sauvegardés dans le fichier : resultats_classification_complets.xlsx


In [44]:
print (resultats_df)

      is_genuine  diagonal  height_left  height_right  margin_low  margin_up  length is_genuine_str  Prediction  Probabilite_Faux (%) Resultat
0           True    171.81       104.86        104.95        4.52       2.89  112.83           Vrai           0                   9.0     VRAI
1           True    171.46       103.36        103.66        3.77       2.99  113.09           Vrai           0                   1.0     VRAI
2           True    172.69       104.48        103.50        4.40       2.94  113.16           Vrai           0                   0.0     VRAI
3           True    171.36       103.91        103.94        3.62       3.01  113.51           Vrai           0                   0.0     VRAI
4           True    171.73       104.28        103.46        4.04       3.48  112.54           Vrai           0                  35.0     VRAI
...          ...       ...          ...           ...         ...        ...     ...            ...         ...                   ...      ...